# TN DCS CANS Assessment Data Analysis

In [11]:
import pandas as pd
import numpy as np 

In [12]:
pd.set_option('display.max_columns', None) 

<h3 style="color:navy">Import Data</h3>

In [13]:
df_youth = pd.read_csv('../data/bdaic created tables/df_youth.csv', dtype={'PERSON ID': str, 'REMOVAL ZIP CODE': str, 'PLACEMENT ZIP CODE': str}, 
parse_dates=['LOCATION BEGIN DATE', 'LOCATION END DATE'])
df_data = pd.read_csv('../data/bdaic created tables/df_assessment_questions.csv', dtype={'PERSON ID': str}) 

In [14]:
### Summary of Youth Data
print('df_youth shape:\n', df_youth.shape)
print('\ndf_youth columns:\n', df_youth.columns)
print('\ndf_youth dtypes:\n', df_youth.dtypes)

df_youth shape:
 (5684, 11)

df_youth columns:
 Index(['PERSON ID', 'CURRENT AGE', 'COMMITMENT COUNTY', 'RESPONSIBLE COUNTY',
       'GENDER', 'REMOVAL ZIP CODE', 'PLACEMENT ZIP CODE',
       'LOCATION BEGIN DATE', 'LOCATION END DATE', 'CANS', 'LIFESKILL'],
      dtype='object')

df_youth dtypes:
 PERSON ID                      object
CURRENT AGE                     int64
COMMITMENT COUNTY              object
RESPONSIBLE COUNTY             object
GENDER                         object
REMOVAL ZIP CODE               object
PLACEMENT ZIP CODE             object
LOCATION BEGIN DATE    datetime64[ns]
LOCATION END DATE      datetime64[ns]
CANS                          float64
LIFESKILL                     float64
dtype: object


In [15]:
df_youth.head()

,PERSON ID,CURRENT AGE,COMMITMENT COUNTY,RESPONSIBLE COUNTY,GENDER,REMOVAL ZIP CODE,PLACEMENT ZIP CODE,LOCATION BEGIN DATE,LOCATION END DATE,CANS,LIFESKILL
0,353258,17,Madison,Madison,MALE,37040,37040,2023-06-13,2024-05-25,1.0,0.0
1,706512,17,Sullivan,Sullivan,FEMALE,37642,37642,2024-11-18,NaT,1.0,0.0
2,1348374,17,Warren,Warren,FEMALE,37110,37110,2023-06-02,2023-09-30,1.0,0.0
3,2470086,18,Hamilton,Hamilton,MALE,37218,37218,2024-09-17,NaT,1.0,0.0
4,3965628,21,Davidson,Davidson,MALE,37725,37725,2021-12-22,2022-08-31,1.0,0.0


In [16]:
### Summary of Assessment Data
print('df_data shape:\n', df_data.shape)
print('\ndf_data columns:\n', df_data.columns)
print('\ndf_data dtypes:\n', df_data.dtypes)

df_data shape:
 (555168, 5)

df_data columns:
 Index(['PERSON ID', 'ASSESSMENT TYPE', 'ASSESSMENT INDICATOR', 'SCORE-RESULT',
       'GENERAL RESULT DESCRIPTION'],
      dtype='object')

df_data dtypes:
 PERSON ID                     object
ASSESSMENT TYPE               object
ASSESSMENT INDICATOR          object
SCORE-RESULT                  object
GENERAL RESULT DESCRIPTION    object
dtype: object


In [17]:
# Create summary statistics
total_youth = len(df_youth['PERSON ID'].unique())
cans_youth = len(df_youth[df_youth['CANS'] == 1.0]['PERSON ID'].unique())
lifeskill_youth = len(df_youth[df_youth['LIFESKILL'] == 1.0]['PERSON ID'].unique())
gender_counts = df_youth['GENDER'].value_counts()
age_stats = df_youth['CURRENT AGE'].describe()

# Print summary with formatting
print('Summary of Youth Dataset\n' + '='*25)
print(f'\nTotal Unique Youth: {total_youth:,}')
print(f'Youth with CANS Assessment: {cans_youth:,}')
print(f'Youth with LIFESKILL Assessment: {lifeskill_youth:,}')

print('\nGender Breakdown:')
print('-----------------')
for gender, count in gender_counts.items():
    percentage = (count/len(df_youth))*100
    print(f'{gender}: {count:,} ({percentage:.1f}%)')

print('\nAge Distribution:')
print('----------------')
print(f'Average Age: {age_stats["mean"]:.1f}')
print(f'Minimum Age: {age_stats["min"]:.0f}')
print(f'Maximum Age: {age_stats["max"]:.0f}')
print('\nAge Breakdown:')
age_counts = df_youth['CURRENT AGE'].value_counts().sort_index()
for age, count in age_counts.items():
    percentage = (count/len(df_youth))*100
    print(f'Age {age:.0f}: {count:,} ({percentage:.1f}%)')


Summary of Youth Dataset

Total Unique Youth: 5,684
Youth with CANS Assessment: 5,667
Youth with LIFESKILL Assessment: 374

Gender Breakdown:
-----------------
MALE: 3,342 (58.8%)
FEMALE: 2,342 (41.2%)

Age Distribution:
----------------
Average Age: 18.2
Minimum Age: 16
Maximum Age: 21

Age Breakdown:
Age 16: 108 (1.9%)
Age 17: 1,705 (30.0%)
Age 18: 1,630 (28.7%)
Age 19: 1,295 (22.8%)
Age 20: 897 (15.8%)
Age 21: 49 (0.9%)


In [18]:
### Make some formating adustments to df_data 
# Rename assessment type and indicator columns to proper case
df_data = df_data.rename(columns={
    'ASSESSMENT TYPE': 'Assessment Type',
    'ASSESSMENT INDICATOR': 'Assessment Indicator'
})

df_data.head() 

,PERSON ID,Assessment Type,Assessment Indicator,SCORE-RESULT,GENERAL RESULT DESCRIPTION
0,353258,CANS,Sexual Abuse,0,No current need; no need for action or interv...
1,353258,CANS,Physical Abuse,0,No current need; no need for action or interv...
2,353258,CANS,Emotional Abuse,0,No current need; no need for action or interv...
3,353258,CANS,Neglect,2,Action or intervention is required to ensure ...
4,353258,CANS,Medical Trauma,0,No current need; no need for action or interv...


### Categorize CANS Questions

In [19]:
# Get CANS assessment indicators and counts
cans_indicators = df_data[df_data['Assessment Type'] == 'CANS'].groupby('Assessment Indicator')['PERSON ID'].nunique().reset_index()
cans_indicators.columns = ['Assessment Indicator', 'Number of Youth']

# Calculate percentage of youth who completed CANS
cans_indicators['Percent of Total Youth'] = (cans_indicators['Number of Youth'] / cans_youth * 100).round(1)

# Sort by number of youth descending
cans_indicators = cans_indicators.sort_values('Number of Youth', ascending=False)

print("CANS Assessment Indicators Summary:")
print("==================================")
print(cans_indicators.to_string(index=False))


CANS Assessment Indicators Summary:
                         Assessment Indicator  Number of Youth  Percent of Total Youth
                             Medical/Physical             5667                   100.0
                                     Runaway*             5667                   100.0
                             Family Strengths             5667                   100.0
                                 Fire Setting             5667                   100.0
                  Sexually Reactive Behavior*             5667                   100.0
                           Sexual Development             5667                   100.0
                    Impulsivity/Hyperactivity             5667                   100.0
                            Sexual Aggression             5667                   100.0
                      Intentional Misbehavior             5667                   100.0
           Interpersonal/Social Connectedness             5667                   100.0
       

In [20]:
cans_indicators.sort_values('Assessment Indicator').head(25)

,Assessment Indicator,Number of Youth,Percent of Total Youth
0,Adjustment To Trauma,5641,99.5
1,Anger Control,5667,100.0
2,Anxiety,5667,100.0
3,Arrests,3902,68.9
4,Attachment Difficulties,5667,100.0
5,Attention/Concentration,5667,100.0
6,Avoidance,5641,99.5
7,Bullying Others,5667,100.0
8,CSEM Risk Question 1,2081,36.7
9,CSEM Risk Question 2,3305,58.3


In [21]:
# Create dictionary mapping indicators to categories
indicator_categories = {
    # Mental Health indicators
    'Decision-Making (Judgement)': 'Mental Health',             # Behavioral & Impulse Control
    'Impulsivity/Hyperactivity': 'Mental Health',               # Behavioral & Impulse Control
    'School Behavior': 'Mental Health',                         # Behavioral & Impulse Control
    'Substance Use': 'Mental Health',                           # Behavioral & Impulse Control
    'Spiritual/Religious': 'Mental Health',                     # Cultural, Spiritual, and Engagement Factors
    'Traditions And Rituals': 'Mental Health',                  # Cultural, Spiritual, and Engagement Factors
    'Youth Involvement With Care': 'Mental Health',             # Cultural, Spiritual, and Engagement Factors
    'Anger Control': 'Mental Health',                           # Mood & Emotional Regulation 
    'Anxiety': 'Mental Health',                                 # Mood & Emotional Regulation 
    'Depression': 'Mental Health',                              # Mood & Emotional Regulation 
    'Emotional And/Or Physical Dysregulation': 'Mental Health', # Mood & Emotional Regulation 
    'Numbing': 'Mental Health',                                 # Mood & Emotional Regulation 
    'Medical/Physical': 'Mental Health',                        # Physical & Medical 
    'Psychosis (Thought Disorder)': 'Mental Health',            # Psychotic & Thought Disturbances
    'Danger To Others': 'Mental Health',                        # Risk & Safety
    'Intentional Misbehavior': 'Mental Health',                 # Risk & Safety  
    'Non-Suicidal Self-Injurious Behavior': 'Mental Health',    # Risk & Safety
    'Other Self-Harm (Recklessness)': 'Mental Health',          # Risk & Safety
    'Suicide Risk': 'Mental Health',                            # Risk & Safety 
    'Sexual Aggression': 'Mental Health',                       # Sexual & Developmental Concerns
    'Sexual Development': 'Mental Health',                      # Sexual & Developmental Concerns
    'Adjustment To Trauma': 'Mental Health',                    # Trauma & Stress-Related
    'Cultural Stress': 'Mental Health',                         # Trauma & Stress-Related
    'Emotional Abuse': 'Mental Health',                         # Trauma & Stress-Related
    'Environmental Influences': 'Mental Health',                # Trauma & Stress-Related
    'Medical Trauma': 'Mental Health',                          # Trauma & Stress-Related
    'Sexual Abuse': 'Mental Health',                            # Trauma & Stress-Related
    
    
    # Supportive Adult Relationships indicators
    'Attachment Difficulties': 'Supportive Adult Relationships',
    'Community Life': 'Supportive Adult Relationships',
    'Family Functioning': 'Supportive Adult Relationships',
    'Family Strengths': 'Supportive Adult Relationships',
    'Interpersonal/Social Connectedness': 'Supportive Adult Relationships',
    'Natural Supports': 'Supportive Adult Relationships',
    'Neglect': 'Supportive Adult Relationships',
    'Oppositional(Non-compliance with Authority)': 'Supportive Adult Relationships',
    'Relationship Permanence': 'Supportive Adult Relationships',
    'Social Functioning': 'Supportive Adult Relationships',
    
    # Housing Instability indicators
    'Independent Living Skills': 'Housing',
    'Living Situation': 'Housing Instability', 
    'Physical Abuse': 'Housing Instability',
    'Runaway*': 'Housing Instability',
    'Youth Residential Stability': 'Housing Instability'
}

# Add category column
cans_indicators['Pillar'] = cans_indicators['Assessment Indicator'].map(indicator_categories)

# Display categorized indicators
print("\nCategorized CANS Assessment Indicators:")
print("======================================")
for category in ['Mental Health', 'Supportive Adult Relationships', 'Housing Instability']:
    print(f"\n{category}:")
    print(cans_indicators[cans_indicators['Pillar'] == category][['Assessment Indicator', 'Number of Youth']].to_string(index=False))



Categorized CANS Assessment Indicators:

Mental Health:
                   Assessment Indicator  Number of Youth
                       Medical/Physical             5667
                     Sexual Development             5667
              Impulsivity/Hyperactivity             5667
                      Sexual Aggression             5667
                Intentional Misbehavior             5667
                           Sexual Abuse             5667
                        School Behavior             5667
                         Medical Trauma             5667
                          Anger Control             5667
   Non-Suicidal Self-Injurious Behavior             5667
         Other Self-Harm (Recklessness)             5667
                        Emotional Abuse             5667
                                Anxiety             5667
                 Traditions And Rituals             5667
                           Suicide Risk             5667
                          Subst

In [22]:
### Filter CANS indicators to pillar-related indicators only 
pillar_indicators = cans_indicators[cans_indicators['Pillar'].notna()]

print(pillar_indicators.shape)
pillar_indicators.head()

(42, 4)


,Assessment Indicator,Number of Youth,Percent of Total Youth,Pillar
50,Medical/Physical,5667,100.0,Mental Health
69,Runaway*,5667,100.0,Housing Instability
33,Family Strengths,5667,100.0,Supportive Adult Relationships
78,Sexual Development,5667,100.0,Mental Health
37,Impulsivity/Hyperactivity,5667,100.0,Mental Health


In [23]:
   Behavioral_impulse_control ={ 'Decision-Making (Judgement)': 'Mental Health',             # Behavioral & Impulse Control
    'Impulsivity/Hyperactivity': 'Mental Health',               # Behavioral & Impulse Control
    'School Behavior': 'Mental Health',                         # Behavioral & Impulse Control
    'Substance Use': 'Mental Health',  # Behavioral & Impulse Control
    'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' :  'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' :  'RESPONSIBLE COUNTY',
   'Region' : 'Region'

   }
   Cultural_spiritual_engagement ={ 'Spiritual/Religious': 'Mental Health',                     # Cultural, Spiritual, and Engagement Factors
    'Traditions And Rituals': 'Mental Health',                  # Cultural, Spiritual, and Engagement Factors
    'Youth Involvement With Care': 'Mental Health' ,            # Cultural, Spiritual, and Engagement Factors
    'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' :  'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' :  'RESPONSIBLE COUNTY',
   'Region' : 'Region'
   }
   Mood_emotional_regulation ={ 'Anger Control': 'Mental Health',                           # Mood & Emotional Regulation 
    'Anxiety': 'Mental Health',                                 # Mood & Emotional Regulation 
    'Depression': 'Mental Health',                              # Mood & Emotional Regulation 
    'Emotional And/Or Physical Dysregulation': 'Mental Health', # Mood & Emotional Regulation 
    'Numbing': 'Mental Health' ,                               # Mood & Emotional Regulation 
    'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' :  'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' :  'RESPONSIBLE COUNTY',
   'Region' : 'Region'
   }
   Physical_medical ={ 'Medical/Physical': 'Mental Health',                        # Physical & Medical 
   'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' :  'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' :  'RESPONSIBLE COUNTY',
   'Region' : 'Region'
   }
   Psychotic_thought_disturbances ={ 'Psychosis (Thought Disorder)': 'Mental Health'  ,     # Psychotic & Thought Disturbances
   'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' :  'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' :  'RESPONSIBLE COUNTY',
   'Region' : 'Region'
   }

Risk_safety ={ 'Danger To Others': 'Mental Health',       # Risk & Safety
    'Intentional Misbehavior': 'Mental Health',                 # Risk & Safety  
    'Non-Suicidal Self-Injurious Behavior': 'Mental Health',    # Risk & Safety
    'Other Self-Harm (Recklessness)': 'Mental Health',          # Risk & Safety
    'Suicide Risk': 'Mental Health',                            # Risk & Safety 
    'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' :  'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' :  'RESPONSIBLE COUNTY',
   'Region' : 'Region'
    }
Sexual_developmental_concerns ={ 'Sexual Aggression': 'Mental Health',                       # Sexual & Developmental Concerns
    'Sexual Development': 'Mental Health',                      # Sexual & Developmental Concerns
    'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' :  'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' :  'RESPONSIBLE COUNTY',
   'Region' : 'Region'
    }
Trauma_stress_related ={ 'Adjustment To Trauma': 'Mental Health',                    # Trauma & Stress-Related
    'Cultural Stress': 'Mental Health',                         # Trauma & Stress-Related
    'Emotional Abuse': 'Mental Health',                         # Trauma & Stress-Related
    'Environmental Influences': 'Mental Health',                # Trauma & Stress-Related
    'Medical Trauma': 'Mental Health',                          # Trauma & Stress-Related
    'Sexual Abuse': 'Mental Health'  ,                          # Trauma & Stress-Related
    'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' : 'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' : 'RESPONSIBLE COUNTY',
   'Region' : 'Region'
    }

In [24]:
    # Supportive Adult Relationships indicators
    family = {
        'Family Functioning': 'Supportive Adult Relationships',
        'Family Strengths': 'Supportive Adult Relationships',
        'Relationship Permanence': 'Supportive Adult Relationships',
        'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' : 'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' : 'RESPONSIBLE COUNTY',
   'Region' : 'Region'
    }
   
    social = {
        'Community Life': 'Supportive Adult Relationships',
        'Attachment Difficulties': 'Supportive Adult Relationships',
        'Interpersonal/Social Connectedness': 'Supportive Adult Relationships',
        'Natural Supports': 'Supportive Adult Relationships',
        'Social Functioning': 'Supportive Adult Relationships',
        'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' : 'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' : 'RESPONSIBLE COUNTY',
   'Region' : 'Region'
    }

    barriers = {
        'Oppositional(Non-compliance with Authority)': 'Supportive Adult Relationships',
        'Neglect': 'Supportive Adult Relationships',
        'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' : 'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' : 'RESPONSIBLE COUNTY',
   'Region' : 'Region'
    }
    skills = {
        'Independent Living Skills': 'Supportive Adult Relationships',
        'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' : 'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' : 'RESPONSIBLE COUNTY',
   'Region' : 'Region'
    }
    living = {
        'Living Situation': 'Housing Instability',
        'Youth Residential Stability': 'Housing Instability',
        'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' : 'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' : 'RESPONSIBLE COUNTY',
   'Region' : 'Region'
    }
    physical_risk = {
        'Physical Abuse': 'Housing Instability',
        'Runaway*': 'Housing Instability',
        'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' : 'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' : 'RESPONSIBLE COUNTY',
   'Region' : 'Region'
    }
    stability = {
    'Skills & Readiness': 'Housing Instability',
    'Current Living Situation': 'Housing Instability',
    'Barriers & Risks': 'Housing Instability',
    'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' : 'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' : 'RESPONSIBLE COUNTY',
   'Region' : 'Region'
}
    
    
    
    
    
    
    
    

<h3 style="color:navy">Filter CANS indicators to pillar-related indicators only</h3>

In [25]:
df_pillar = df_data.merge(pillar_indicators[['Assessment Indicator', 'Pillar']], how='inner', on='Assessment Indicator') 

print(df_pillar.shape)
df_pillar.head()


(232744, 6)


,PERSON ID,Assessment Type,Assessment Indicator,SCORE-RESULT,GENERAL RESULT DESCRIPTION,Pillar
0,353258,CANS,Sexual Abuse,0,No current need; no need for action or interv...,Mental Health
1,706512,CANS,Sexual Abuse,1,"Identified need requires monitoring, watchful...",Mental Health
2,1348374,CANS,Sexual Abuse,2,Action or intervention is required to ensure ...,Mental Health
3,2470086,CANS,Sexual Abuse,0,No current need; no need for action or interv...,Mental Health
4,3965628,CANS,Sexual Abuse,0,No current need; no need for action or interv...,Mental Health


In [26]:
df_pillar_pivot = df_pillar.pivot_table(index='PERSON ID', columns='Assessment Indicator', values='SCORE-RESULT', aggfunc='first')

print(df_pillar_pivot.shape)
df_pillar_pivot.head()




(5667, 42)


Assessment Indicator,Adjustment To Trauma,Anger Control,Anxiety,Attachment Difficulties,Community Life,Cultural Stress,Danger To Others,Decision-Making (Judgement),Depression,Emotional Abuse,Emotional And/Or Physical Dysregulation,Environmental Influences,Family Functioning,Family Strengths,Impulsivity/Hyperactivity,Independent Living Skills,Intentional Misbehavior,Interpersonal/Social Connectedness,Living Situation,Medical Trauma,Medical/Physical,Natural Supports,Neglect,Non-Suicidal Self-Injurious Behavior,Numbing,Oppositional(Non-compliance with Authority),Other Self-Harm (Recklessness),Physical Abuse,Psychosis (Thought Disorder),Relationship Permanence,Runaway*,School Behavior,Sexual Abuse,Sexual Aggression,Sexual Development,Social Functioning,Spiritual/Religious,Substance Use,Suicide Risk,Traditions And Rituals,Youth Involvement With Care,Youth Residential Stability
PERSON ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10003244,0,0,0,0,3,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,1,0,0,0,0
10004996,1,1,2,1,2,0,1,1,2,1,1,0,1,2,2,1,1,2,1,0,1,3,2,1,1,1,1,2,0,2,0,1,1,0,1,2,3,0,1,0,0,1
100079976,1,1,2,1,3,0,1,2,0,0,2,0,1,3,2,0,1,2,1,0,0,3,0,1,1,1,1,0,0,2,1,1,2,0,1,2,3,0,0,0,1,0
10010134,2,0,1,0,3,0,0,1,1,1,0,0,2,2,1,0,0,0,0,0,0,1,3,1,0,1,0,1,0,2,0,0,0,0,0,0,3,1,1,0,0,1
100106936,1,1,1,0,2,0,0,1,1,0,1,1,1,2,1,1,0,1,0,0,0,1,3,0,0,1,1,0,0,1,0,0,0,0,2,1,2,1,1,0,0,1


<h3 style="color:navy">Merge df_youth with df_pillar_pivot</h3>

In [27]:
df_cans = df_youth.merge(df_pillar_pivot, how='inner', on='PERSON ID').drop(columns=['CANS', 'LIFESKILL'])

print(df_cans.shape)
df_cans.head()

(5667, 51)


,PERSON ID,CURRENT AGE,COMMITMENT COUNTY,RESPONSIBLE COUNTY,GENDER,REMOVAL ZIP CODE,PLACEMENT ZIP CODE,LOCATION BEGIN DATE,LOCATION END DATE,Adjustment To Trauma,Anger Control,Anxiety,Attachment Difficulties,Community Life,Cultural Stress,Danger To Others,Decision-Making (Judgement),Depression,Emotional Abuse,Emotional And/Or Physical Dysregulation,Environmental Influences,Family Functioning,Family Strengths,Impulsivity/Hyperactivity,Independent Living Skills,Intentional Misbehavior,Interpersonal/Social Connectedness,Living Situation,Medical Trauma,Medical/Physical,Natural Supports,Neglect,Non-Suicidal Self-Injurious Behavior,Numbing,Oppositional(Non-compliance with Authority),Other Self-Harm (Recklessness),Physical Abuse,Psychosis (Thought Disorder),Relationship Permanence,Runaway*,School Behavior,Sexual Abuse,Sexual Aggression,Sexual Development,Social Functioning,Spiritual/Religious,Substance Use,Suicide Risk,Traditions And Rituals,Youth Involvement With Care,Youth Residential Stability
0,353258,17,Madison,Madison,MALE,37040,37040,2023-06-13,2024-05-25,1,0,1,0,2,0,0,1,1,0,1,NaN,1,1,0,0,1,1,1,0,0,2,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
1,706512,17,Sullivan,Sullivan,FEMALE,37642,37642,2024-11-18,NaT,1,0,0,1,1,0,0,1,0,0,0,NaN,1,2,0,0,0,1,0,1,0,3,3,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0
2,1348374,17,Warren,Warren,FEMALE,37110,37110,2023-06-02,2023-09-30,1,1,1,1,3,0,0,1,1,0,0,0,1,0,1,1,1,1,0,0,0,1,0,1,0,1,0,0,0,0,0,1,2,0,0,1,1,1,1,0,0,0
3,2470086,18,Hamilton,Hamilton,MALE,37218,37218,2024-09-17,NaT,0,2,2,0,2,0,2,2,2,0,0,0,1,0,0,2,0,2,1,0,1,3,1,0,0,0,1,0,1,2,1,1,0,0,0,2,3,2,1,0,1,2
4,3965628,21,Davidson,Davidson,MALE,37725,37725,2021-12-22,2022-08-31,2,0,1,1,1,0,2,2,1,0,0,2,0,0,2,0,1,0,0,1,2,0,1,0,0,3,3,0,0,0,0,1,0,0,0,1,0,2,0,0,1,0


In [28]:
# Convert indicator_category keys to list
indicator_cols = list(indicator_categories.keys())


### Reorder columns
# Get first 9 columns
first_cols = df_cans.columns[:9]

# Create ordered column list 
ordered_cols = list(first_cols) + indicator_cols

# Reorder dataframe, keeping only columns that exist
df_cans = df_cans[ordered_cols]

print(df_cans.shape)
df_cans.head()


(5667, 51)


,PERSON ID,CURRENT AGE,COMMITMENT COUNTY,RESPONSIBLE COUNTY,GENDER,REMOVAL ZIP CODE,PLACEMENT ZIP CODE,LOCATION BEGIN DATE,LOCATION END DATE,Decision-Making (Judgement),Impulsivity/Hyperactivity,School Behavior,Substance Use,Spiritual/Religious,Traditions And Rituals,Youth Involvement With Care,Anger Control,Anxiety,Depression,Emotional And/Or Physical Dysregulation,Numbing,Medical/Physical,Psychosis (Thought Disorder),Danger To Others,Intentional Misbehavior,Non-Suicidal Self-Injurious Behavior,Other Self-Harm (Recklessness),Suicide Risk,Sexual Aggression,Sexual Development,Adjustment To Trauma,Cultural Stress,Emotional Abuse,Environmental Influences,Medical Trauma,Sexual Abuse,Attachment Difficulties,Community Life,Family Functioning,Family Strengths,Interpersonal/Social Connectedness,Natural Supports,Neglect,Oppositional(Non-compliance with Authority),Relationship Permanence,Social Functioning,Independent Living Skills,Living Situation,Physical Abuse,Runaway*,Youth Residential Stability
0,353258,17,Madison,Madison,MALE,37040,37040,2023-06-13,2024-05-25,1,0,0,1,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,NaN,0,0,0,2,1,1,1,2,2,0,1,0,0,1,0,0,0
1,706512,17,Sullivan,Sullivan,FEMALE,37642,37642,2024-11-18,NaT,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,NaN,1,1,1,1,1,2,1,3,3,0,1,0,0,0,0,0,0
2,1348374,17,Warren,Warren,FEMALE,37110,37110,2023-06-02,2023-09-30,1,1,1,1,1,0,0,1,1,1,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,2,1,3,1,0,1,1,0,1,0,1,1,0,0,0,0
3,2470086,18,Hamilton,Hamilton,MALE,37218,37218,2024-09-17,NaT,2,0,1,2,3,0,1,2,2,2,0,0,1,1,2,0,0,1,1,0,0,0,0,0,0,0,0,0,2,1,0,2,3,1,0,2,2,2,1,0,1,2
4,3965628,21,Davidson,Davidson,MALE,37725,37725,2021-12-22,2022-08-31,2,2,1,2,0,0,1,0,1,1,0,0,2,0,2,1,0,3,0,0,0,2,0,0,2,1,0,1,1,0,0,0,0,1,3,0,1,0,0,0,0,0


In [29]:
tn_counties_by_region = {
    "East": [
        "Anderson", "Bledsoe", "Blount", "Bradley", "Campbell", "Carter",
        "Claiborne", "Cocke", "Cumberland", "Grainger", "Greene", "Hamblen",
        "Hamilton", "Hancock", "Hawkins", "Jefferson", "Johnson", "Knox",
        "Loudon", "Marion", "McMinn", "Meigs", "Monroe", "Morgan", "Polk",
        "Rhea", "Roane", "Scott", "Sevier", "Sullivan", "Unicoi", "Union",
        "Washington"
    ],
    "Middle": [
        "Bedford", "Cannon", "Cheatham", "Clay", "Coffee", "Davidson",
        "DeKalb", "Dickson", "Fentress", "Franklin", "Giles", "Grundy",
        "Hickman", "Houston", "Humphreys", "Jackson", "Lawrence", "Lewis",
        "Lincoln", "Macon", "Marshall", "Maury", "Moore", "Overton", "Perry",
        "Pickett", "Putnam", "Robertson", "Rutherford", "Sequatchie", "Smith",
        "Stewart", "Sumner", "Trousdale", "Van Buren", "Warren", "Wayne",
        "White", "Williamson", "Wilson"
    ],
    "West": [
        "Benton", "Carroll", "Chester", "Crockett", "Decatur", "Dyer",
        "Fayette", "Gibson", "Hardeman", "Hardin", "Haywood", "Henderson",
        "Henry", "Lake", "Lauderdale", "Madison", "McNairy", "Obion",
        "Shelby", "Tipton", "Weakley"
    ]
}


In [30]:
county_to_region = {
    county: region
    for region, counties in tn_counties_by_region.items()
    for county in counties
}

df_cans['Region'] = (
    df_cans['RESPONSIBLE COUNTY'].astype(str).str.strip().map(county_to_region)
)
df_cans['Region'] = df_cans['Region'].fillna('Unknown')

In [31]:
df_cans.head()

,PERSON ID,CURRENT AGE,COMMITMENT COUNTY,RESPONSIBLE COUNTY,GENDER,REMOVAL ZIP CODE,PLACEMENT ZIP CODE,LOCATION BEGIN DATE,LOCATION END DATE,Decision-Making (Judgement),Impulsivity/Hyperactivity,School Behavior,Substance Use,Spiritual/Religious,Traditions And Rituals,Youth Involvement With Care,Anger Control,Anxiety,Depression,Emotional And/Or Physical Dysregulation,Numbing,Medical/Physical,Psychosis (Thought Disorder),Danger To Others,Intentional Misbehavior,Non-Suicidal Self-Injurious Behavior,Other Self-Harm (Recklessness),Suicide Risk,Sexual Aggression,Sexual Development,Adjustment To Trauma,Cultural Stress,Emotional Abuse,Environmental Influences,Medical Trauma,Sexual Abuse,Attachment Difficulties,Community Life,Family Functioning,Family Strengths,Interpersonal/Social Connectedness,Natural Supports,Neglect,Oppositional(Non-compliance with Authority),Relationship Permanence,Social Functioning,Independent Living Skills,Living Situation,Physical Abuse,Runaway*,Youth Residential Stability,Region
0,353258,17,Madison,Madison,MALE,37040,37040,2023-06-13,2024-05-25,1,0,0,1,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,NaN,0,0,0,2,1,1,1,2,2,0,1,0,0,1,0,0,0,West
1,706512,17,Sullivan,Sullivan,FEMALE,37642,37642,2024-11-18,NaT,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,NaN,1,1,1,1,1,2,1,3,3,0,1,0,0,0,0,0,0,East
2,1348374,17,Warren,Warren,FEMALE,37110,37110,2023-06-02,2023-09-30,1,1,1,1,1,0,0,1,1,1,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,2,1,3,1,0,1,1,0,1,0,1,1,0,0,0,0,Middle
3,2470086,18,Hamilton,Hamilton,MALE,37218,37218,2024-09-17,NaT,2,0,1,2,3,0,1,2,2,2,0,0,1,1,2,0,0,1,1,0,0,0,0,0,0,0,0,0,2,1,0,2,3,1,0,2,2,2,1,0,1,2,East
4,3965628,21,Davidson,Davidson,MALE,37725,37725,2021-12-22,2022-08-31,2,2,1,2,0,0,1,0,1,1,0,0,2,0,2,1,0,3,0,0,0,2,0,0,2,1,0,1,1,0,0,0,0,1,3,0,1,0,0,0,0,0,Middle


## Risk greater than 2 for all

### Behvioral Impulse

In [32]:
rename_map = Behavioral_impulse_control
cols_to_keep = list(rename_map.keys())
df_bic = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=rename_map).copy()

In [33]:
df_bic.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Mental Health,Mental Health,Mental Health,Mental Health,Region
0,353258,Madison,MALE,37040,1,0,0,1,West
1,706512,Sullivan,FEMALE,37642,1,0,0,0,East
2,1348374,Warren,FEMALE,37110,1,1,1,1,Middle
3,2470086,Hamilton,MALE,37218,2,0,1,2,East
4,3965628,Davidson,MALE,37725,2,2,1,2,Middle


In [65]:
# --- Select Mental Health Columns ---
bic_cols = df_bic.filter(regex=r'^Mental Health$').columns
nums_bic = df_bic[bic_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask_bic = nums_bic.ge(2).any(axis=1)
df_bic = df_bic[mask_bic]

# --- Build New Dataset with PERSON ID, ZIP, REGION, GENDER ---
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
bic_cols_all = df_bic.filter(regex=r'^Mental Health$').columns
nums = df_bic[bic_cols_all].apply(pd.to_numeric, errors='coerce')
mask = nums.ge(2).any(axis=1)

new_df_bic = df_bic[base_cols].copy()
new_df_bic['MentalHealth_Behavioral_Impulse'] = mask.astype(int)  # 1 if condition met, 0 if not

### Cultural/Spiritual bucket

In [34]:
cols_to_keep = list(Cultural_spiritual_engagement .keys())
df_cse = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=Cultural_spiritual_engagement).copy()

In [35]:
df_cse.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Mental Health,Mental Health,Mental Health,Region
0,353258,Madison,MALE,37040,0,0,0,West
1,706512,Sullivan,FEMALE,37642,1,0,0,East
2,1348374,Warren,FEMALE,37110,1,0,0,Middle
3,2470086,Hamilton,MALE,37218,3,0,1,East
4,3965628,Davidson,MALE,37725,0,0,1,Middle


In [66]:
# --- Select Mental Health Columns ---
cse_cols = df_cse.filter(regex=r'^Mental Health$').columns
nums_cse = df_cse[cse_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask_cse = nums_cse.ge(2).any(axis=1)
df_cse = df_cse[mask_cse]

# --- Build New Dataset with PERSON ID, ZIP, REGION, GENDER ---
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
cse_cols_all = df_cse.filter(regex=r'^Mental Health$').columns
nums = df_cse[cse_cols_all].apply(pd.to_numeric, errors='coerce')
mask = nums.ge(2).any(axis=1)

new_df_cse = df_cse[base_cols].copy()
new_df_cse['MentalHealth_Cultural_Spiritual'] = mask.astype(int)  # 1 if condition met, 0 if not

### Mood/emotional regulation

In [36]:
cols_to_keep = list(Mood_emotional_regulation.keys())
df_mer = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=Mood_emotional_regulation).copy()

In [37]:
df_mer.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Mental Health,Mental Health,Mental Health,Mental Health,Mental Health,Region
0,353258,Madison,MALE,37040,0,1,1,1,0,West
1,706512,Sullivan,FEMALE,37642,0,0,0,0,0,East
2,1348374,Warren,FEMALE,37110,1,1,1,0,0,Middle
3,2470086,Hamilton,MALE,37218,2,2,2,0,0,East
4,3965628,Davidson,MALE,37725,0,1,1,0,0,Middle


In [67]:
# --- Select Mental Health Columns ---
mer_cols = df_mer.filter(regex=r'^Mental Health$').columns
nums_mer = df_mer[mer_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask_mer = nums_mer.ge(2).any(axis=1)
df_mer = df_mer[mask_mer]

# --- Build New Dataset with PERSON ID, ZIP, REGION, GENDER ---
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
mer_cols_all = df_mer.filter(regex=r'^Mental Health$').columns
nums = df_mer[mer_cols_all].apply(pd.to_numeric, errors='coerce')
mask = nums.ge(2).any(axis=1)

new_df_mer = df_mer[base_cols].copy()
new_df_mer['MentalHealth_Mood_Emotional_Regulation'] = mask.astype(int)  # 1 if condition met, 0 if not

### Physical_medical


In [38]:
cols_to_keep = list(Physical_medical.keys())
df_pm = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=Physical_medical).copy()

In [39]:
df_pm.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Mental Health,Region
0,353258,Madison,MALE,37040,0,West
1,706512,Sullivan,FEMALE,37642,0,East
2,1348374,Warren,FEMALE,37110,0,Middle
3,2470086,Hamilton,MALE,37218,1,East
4,3965628,Davidson,MALE,37725,2,Middle


In [68]:
# --- Select Mental Health Columns ---
pm_cols = df_pm.filter(regex=r'^Mental Health$').columns
nums_pm = df_pm[pm_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask_pm = nums_pm.ge(2).any(axis=1)
df_pm = df_pm[mask_pm]

# --- Build New Dataset with PERSON ID, ZIP, REGION, GENDER ---
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
pm_cols_all = df_pm.filter(regex=r'^Mental Health$').columns
nums = df_pm[pm_cols_all].apply(pd.to_numeric, errors='coerce')
mask = nums.ge(2).any(axis=1)

new_df_pm = df_pm[base_cols].copy()
new_df_pm['MentalHealth_Physical_Medical'] = mask.astype(int)  # 1 if condition met, 0 if not

### Psychotic_thought_disturbances


In [40]:

cols_to_keep = list(Psychotic_thought_disturbances.keys())
df_ptd = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=Psychotic_thought_disturbances).copy()

In [41]:
df_ptd.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Mental Health,Region
0,353258,Madison,MALE,37040,0,West
1,706512,Sullivan,FEMALE,37642,0,East
2,1348374,Warren,FEMALE,37110,0,Middle
3,2470086,Hamilton,MALE,37218,1,East
4,3965628,Davidson,MALE,37725,0,Middle


In [69]:
# --- Select Mental Health Columns ---
ptd_cols = df_ptd.filter(regex=r'^Mental Health$').columns
nums_ptd = df_ptd[ptd_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask_ptd = nums_ptd.ge(2).any(axis=1)
df_ptd = df_ptd[mask_ptd]

# --- Build New Dataset with PERSON ID, ZIP, REGION, GENDER ---
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
ptd_cols_all = df_ptd.filter(regex=r'^Mental Health$').columns
nums = df_ptd[ptd_cols_all].apply(pd.to_numeric, errors='coerce')
mask = nums.ge(2).any(axis=1)

new_df_ptd = df_ptd[base_cols].copy()
new_df_ptd['MentalHealth_Physical_Medical'] = mask.astype(int)  # 1 if condition met, 0 if not

### Risk_safety


In [42]:
cols_to_keep = list(Risk_safety.keys())
df_rs = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=Risk_safety).copy()

In [43]:
df_rs.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Mental Health,Mental Health,Mental Health,Mental Health,Mental Health,Region
0,353258,Madison,MALE,37040,0,1,0,0,0,West
1,706512,Sullivan,FEMALE,37642,0,0,0,0,0,East
2,1348374,Warren,FEMALE,37110,0,1,1,0,1,Middle
3,2470086,Hamilton,MALE,37218,2,0,0,1,1,East
4,3965628,Davidson,MALE,37725,2,1,0,3,0,Middle


In [71]:
# --- Select Mental Health Columns ---
rs_cols = df_rs.filter(regex=r'^Mental Health$').columns
nums_rs = df_rs[rs_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask_rs = nums_rs.ge(2).any(axis=1)
df_rs = df_rs[mask_rs]

# --- Build New Dataset with PERSON ID, ZIP, REGION, GENDER ---
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
rs_cols_all = df_rs.filter(regex=r'^Mental Health$').columns
nums = df_rs[rs_cols_all].apply(pd.to_numeric, errors='coerce')
mask = nums.ge(2).any(axis=1)

new_df_rs = df_rs[base_cols].copy()
new_df_rs['MentalHealth_Risk_Safty'] = mask.astype(int)  # 1 if condition met, 0 if not

### Sexual_developmental_concerns

In [44]:
cols_to_keep = list(Sexual_developmental_concerns.keys())
df_sdc = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=Sexual_developmental_concerns).copy()

In [45]:
df_sdc.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Mental Health,Mental Health,Region
0,353258,Madison,MALE,37040,0,0,West
1,706512,Sullivan,FEMALE,37642,0,1,East
2,1348374,Warren,FEMALE,37110,0,0,Middle
3,2470086,Hamilton,MALE,37218,0,0,East
4,3965628,Davidson,MALE,37725,0,0,Middle


In [72]:
# --- Select Mental Health Columns ---
sdc_cols = df_sdc.filter(regex=r'^Mental Health$').columns
nums_sdc = df_sdc[sdc_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask_sdc = nums_sdc.ge(2).any(axis=1)
df_sdc = df_sdc[mask_sdc]

# --- Build New Dataset with PERSON ID, ZIP, REGION, GENDER ---
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
sdc_cols_all = df_sdc.filter(regex=r'^Mental Health$').columns
nums = df_sdc[sdc_cols_all].apply(pd.to_numeric, errors='coerce')
mask = nums.ge(2).any(axis=1)

new_df_sdc = df_sdc[base_cols].copy()
new_df_sdc['MentalHealth_Sexual_devel'] = mask.astype(int)  # 1 if condition met, 0 if not

### Trauma_stress_related

In [46]:
cols_to_keep = list(Trauma_stress_related.keys())
df_tsr = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=Trauma_stress_related).copy()

In [47]:
df_tsr.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Mental Health,Mental Health,Mental Health,Mental Health,Mental Health,Mental Health,Region
0,353258,Madison,MALE,37040,1,0,0,NaN,0,0,West
1,706512,Sullivan,FEMALE,37642,1,0,0,NaN,1,1,East
2,1348374,Warren,FEMALE,37110,1,0,0,0,0,2,Middle
3,2470086,Hamilton,MALE,37218,0,0,0,0,0,0,East
4,3965628,Davidson,MALE,37725,2,0,0,2,1,0,Middle


In [73]:
# --- Select Mental Health Columns ---
tsr_cols = df_tsr.filter(regex=r'^Mental Health$').columns
nums_tsr = df_tsr[tsr_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask_tsr = nums_tsr.ge(2).any(axis=1)
df_tsr = df_tsr[mask_tsr]

# --- Build New Dataset with PERSON ID, ZIP, REGION, GENDER ---
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
tsr_cols_all = df_tsr.filter(regex=r'^Mental Health$').columns
nums = df_tsr[tsr_cols_all].apply(pd.to_numeric, errors='coerce')
mask = nums.ge(2).any(axis=1)

new_df_tsr = df_tsr[base_cols].copy()
new_df_tsr['MentalHealth_Trauma_Stress'] = mask.astype(int)  # 1 if condition met, 0 if not

### Family Relationships

In [50]:
cols_to_keep = list(family .keys())
df_fas = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=family).copy()

In [51]:
df_fas.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Supportive Adult Relationships,Supportive Adult Relationships,Supportive Adult Relationships,Region
0,353258,Madison,MALE,37040,1,1,1,West
1,706512,Sullivan,FEMALE,37642,1,2,1,East
2,1348374,Warren,FEMALE,37110,1,0,0,Middle
3,2470086,Hamilton,MALE,37218,1,0,2,East
4,3965628,Davidson,MALE,37725,0,0,0,Middle


In [75]:
# --- Select Mental Health Columns ---
fas_cols = df_fas.filter(regex=r'^Supportive Adult Relationships$').columns
nums_fas = df_fas[fas_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask_fas = nums_fas.ge(2).any(axis=1)
df_fas = df_fas[mask_fas]

# --- Build New Dataset with PERSON ID, ZIP, REGION, GENDER ---
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
fas_cols_all = df_fas.filter(regex=r'^Supportive Adult Relationships$').columns
nums = df_fas[fas_cols_all].apply(pd.to_numeric, errors='coerce')
mask = nums.ge(2).any(axis=1)

new_df_fas = df_fas[base_cols].copy()
new_df_fas['Supportive_adult_Family_relationships'] = mask.astype(int)  # 1 if condition met, 0 if not

### Social relationships

In [52]:
cols_to_keep = list(social .keys())
df_sas = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=social).copy()

In [53]:
df_sas.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Supportive Adult Relationships,Supportive Adult Relationships,Supportive Adult Relationships,Supportive Adult Relationships,Supportive Adult Relationships,Region
0,353258,Madison,MALE,37040,0,2,1,2,0,West
1,706512,Sullivan,FEMALE,37642,1,1,1,3,0,East
2,1348374,Warren,FEMALE,37110,1,3,1,1,1,Middle
3,2470086,Hamilton,MALE,37218,0,2,2,3,2,East
4,3965628,Davidson,MALE,37725,1,1,0,0,1,Middle


In [77]:
# --- Select Mental Health Columns ---
sas_cols = df_sas.filter(regex=r'^Supportive Adult Relationships$').columns
nums_sas = df_sas[sas_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask_sas = nums_sas.ge(2).any(axis=1)
df_sas = df_sas[mask_sas]

# --- Build New Dataset with PERSON ID, ZIP, REGION, GENDER ---
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
sas_cols_all = df_sas.filter(regex=r'^Supportive Adult Relationships$').columns
nums = df_sas[sas_cols_all].apply(pd.to_numeric, errors='coerce')
mask = nums.ge(2).any(axis=1)

new_df_sas = df_sas[base_cols].copy()
new_df_sas['Supportive_adult_Social_relationships'] = mask.astype(int)  # 1 if condition met, 0 if not

In [54]:
cols_to_keep = list(barriers .keys())
df_bas = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=barriers).copy()

Barriers to support

In [55]:
df_bas.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Supportive Adult Relationships,Supportive Adult Relationships,Region
0,353258,Madison,MALE,37040,2,0,West
1,706512,Sullivan,FEMALE,37642,3,0,East
2,1348374,Warren,FEMALE,37110,0,1,Middle
3,2470086,Hamilton,MALE,37218,1,0,East
4,3965628,Davidson,MALE,37725,1,3,Middle


In [78]:
# --- Select Mental Health Columns ---
bas_cols = df_bas.filter(regex=r'^Supportive Adult Relationships$').columns
nums_bas = df_bas[bas_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask_bas = nums_bas.ge(2).any(axis=1)
df_bas = df_bas[mask_bas]

# --- Build New Dataset with PERSON ID, ZIP, REGION, GENDER ---
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
bas_cols_all = df_bas.filter(regex=r'^Supportive Adult Relationships$').columns
nums = df_bas[bas_cols_all].apply(pd.to_numeric, errors='coerce')
mask = nums.ge(2).any(axis=1)

new_df_bas = df_bas[base_cols].copy()
new_df_bas['Supportive_adult_Social_relationships'] = mask.astype(int)  # 1 if condition met, 0 if not

### Housing

Skills Readiness

In [56]:
cols_to_keep = list(skills .keys())
df_skas = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=skills).copy()

In [57]:
df_skas.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Supportive Adult Relationships,Region
0,353258,Madison,MALE,37040,0,West
1,706512,Sullivan,FEMALE,37642,0,East
2,1348374,Warren,FEMALE,37110,1,Middle
3,2470086,Hamilton,MALE,37218,2,East
4,3965628,Davidson,MALE,37725,0,Middle


In [79]:
# --- Select Mental Health Columns ---
skas_cols = df_skas.filter(regex=r'^Supportive Adult Relationships$').columns
nums_skas = df_skas[skas_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask_skas = nums_skas.ge(2).any(axis=1)
df_skas = df_skas[mask_skas]

# --- Build New Dataset with PERSON ID, ZIP, REGION, GENDER ---
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
skas_cols_all = df_skas.filter(regex=r'^Supportive Adult Relationships$').columns
nums = df_skas[skas_cols_all].apply(pd.to_numeric, errors='coerce')
mask = nums.ge(2).any(axis=1)

new_df_skas = df_skas[base_cols].copy()
new_df_skas['Housing_instability_Skills'] = mask.astype(int)  # 1 if condition met, 0 if not

Current living

In [58]:
cols_to_keep = list(living .keys())
df_las = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=living).copy()

In [59]:
df_las.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Housing Instability,Housing Instability,Region
0,353258,Madison,MALE,37040,1,0,West
1,706512,Sullivan,FEMALE,37642,0,0,East
2,1348374,Warren,FEMALE,37110,0,0,Middle
3,2470086,Hamilton,MALE,37218,1,2,East
4,3965628,Davidson,MALE,37725,0,0,Middle


In [80]:
# --- Select Mental Health Columns ---
las_cols = df_las.filter(regex=r'^Housing Instability$').columns
nums_las = df_las[las_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask_las = nums_las.ge(2).any(axis=1)
df_las = df_las[mask_las]

# --- Build New Dataset with PERSON ID, ZIP, REGION, GENDER ---
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
las_cols_all = df_las.filter(regex=r'^Housing Instability$').columns
nums = df_las[las_cols_all].apply(pd.to_numeric, errors='coerce')
mask = nums.ge(2).any(axis=1)

new_df_las = df_las[base_cols].copy()
new_df_las['Housing_instability_Current_living'] = mask.astype(int)  # 1 if condition met, 0 if not

Barriers Risk

In [60]:
cols_to_keep = list(physical_risk .keys())
df_pras = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=physical_risk).copy()

In [61]:
df_pras.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Housing Instability,Housing Instability,Region
0,353258,Madison,MALE,37040,0,0,West
1,706512,Sullivan,FEMALE,37642,0,0,East
2,1348374,Warren,FEMALE,37110,0,0,Middle
3,2470086,Hamilton,MALE,37218,0,1,East
4,3965628,Davidson,MALE,37725,0,0,Middle


In [81]:
# --- Select Mental Health Columns ---
pras_cols = df_pras.filter(regex=r'^Housing Instability$').columns
nums_pras = df_pras[pras_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask_pras = nums_pras.ge(2).any(axis=1)
df_pras = df_pras[mask_pras]

# --- Build New Dataset with PERSON ID, ZIP, REGION, GENDER ---
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
pras_cols_all = df_pras.filter(regex=r'^Housing Instability$').columns
nums = df_pras[pras_cols_all].apply(pd.to_numeric, errors='coerce')
mask = nums.ge(2).any(axis=1)

new_df_pras = df_pras[base_cols].copy()
new_df_pras['Housing_instability_Barriers'] = mask.astype(int)  # 1 if condition met, 0 if not

## One Df

In [82]:

from functools import reduce

# Put all your dataframes in a list
dfs = [
    new_df_bic,
    new_df_cse,
    new_df_mer,
    new_df_pm,
    new_df_ptd,
    new_df_rs,
    new_df_sdc,
    new_df_tsr,
    new_df_fas,
    new_df_sas,
    new_df_bas,
    new_df_skas,
    new_df_las,
    new_df_pras
]

# Merge all on PERSON ID
merged_df = reduce(lambda left, right: pd.merge(left, right, on="PERSON ID", how="outer"), dfs)


C:\Users\emros\AppData\Local\Temp\ipykernel_24356\4165365159.py:22: FutureWarning: Passing 'suffixes' which cause duplicate columns {'GENDER_x', 'PLACEMENT ZIP CODE_x', 'Region_x'} in the result is deprecated and will raise a MergeError in a future version.
  merged_df = reduce(lambda left, right: pd.merge(left, right, on="PERSON ID", how="outer"), dfs)
C:\Users\emros\AppData\Local\Temp\ipykernel_24356\4165365159.py:22: FutureWarning: Passing 'suffixes' which cause duplicate columns {'GENDER_x', 'PLACEMENT ZIP CODE_x', 'Region_x'} in the result is deprecated and will raise a MergeError in a future version.
  merged_df = reduce(lambda left, right: pd.merge(left, right, on="PERSON ID", how="outer"), dfs)
C:\Users\emros\AppData\Local\Temp\ipykernel_24356\4165365159.py:22: FutureWarning: Passing 'suffixes' which cause duplicate columns {'GENDER_x', 'PLACEMENT ZIP CODE_x', 'Region_x'} in the result is deprecated and will raise a MergeError in a future version.
  merged_df = reduce(lambda le

In [83]:
merged_df.to_csv("Risk_data.csv", index=False)

In [87]:
merged_df.to_csv(r"C:\Users\emros\Downloads\Risk_data.csv", index=False)
